In [8]:
import gzip
import requests
import random
import numpy as np
import pandas as pd

r/news:

In [ ]:
sub = 'news'
size = 500
after = [1615766400, # March 15, 2021 12:00:00 AM GMT
         1611964800, # January 30, 2021 12:00:00 AM GMT
         1626566400, # July 18, 2021 12:00:00 AM GMT
         1609891200, # January 6, 2021 12:00:00 AM GMT
         1616198400, # March 20, 2021 12:00:00 AM GMT
         1637452800, # November 21, 2021 12:00:00 AM GMT
         1634774400, # October 21, 2021 12:00:00 AM GMT
         1640390400, # December 25, 2021 12:00:00 AM GMT
         1627516800, # July 29, 2021 12:00:00 AM GMT
         1628380800, # August 8, 2021 12:00:00 AM GMT
         1626220800, # July 14, 2021 12:00:00 AM GMT
         1610582400, # January 14, 2021 12:00:00 AM GMT
         1613779200, # February 20, 2021 12:00:00 AM GMT
         1618704000, # April 18, 2021 12:00:00 AM GMT
         1636156800 # November 6, 2021 12:00:00 AM GMT
         ]

In [ ]:
rnews = pd.DataFrame(columns=['body'])

for epoch in after:
    url = f'https://api.pushshift.io/reddit/search/comment/?fields=body&subreddit={sub}&size={size}&after={epoch}'
    rnews = pd.concat([rnews, pd.DataFrame.from_dict(requests.get(url).json()['data'])], ignore_index=True)

In [ ]:
rnews

In [ ]:
rnews_notdel = rnews.query('body not in ["[removed]", "[deleted]"]').reset_index().drop(columns=['index'])
rnews_notdel

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='rnews_data.csv')
rnews_notdel.to_csv('rnews_data.zip', index=False, compression=compression_opts)

In [ ]:
rnews = pd.read_csv('rnews_data.csv')
rnews

In [ ]:
random.seed(1)
n = rnews.shape[0]
indices = random.sample(range(n), 30)
rnews.iloc[indices].to_csv('rnews_sample30.txt', index=False)

Amazon book reviews:

In [ ]:
with gzip.open('Kindle_Store_5.json.gz', 'rb') as f:
    reviews_df = pd.read_json(f, orient='records', lines=True)

In [ ]:
random.seed(1)
reviews_data = reviews_df[['reviewText']].iloc[random.sample(range(reviews_df.shape[0]), 1000)]
reviews_data

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='reviews_data.csv')
reviews_data.to_csv('reviews_data.zip', index=False, compression=compression_opts)

In [ ]:
random.seed(1)
reviews_data.reset_index().drop(
        columns=['index']
    ).iloc[random.sample(range(1000), 20)].to_csv(
        'reviews_sample30.csv', index=False
    )

Annotation (AP2):

In [ ]:
import zipfile

with zipfile.ZipFile('data/reviews_data.zip', 'r') as z:
    with z.open('reviews_data.csv') as f:
        reviews_data = pd.read_csv(f)

In [ ]:
reviews_data

In [ ]:
reviews_eval = pd.read_csv('data/reviews_eval_adj.csv')
reviews_exp = pd.read_csv('data/reviews_exp_adj.csv').convert_dtypes()

In [ ]:
display(reviews_eval.head())
display(reviews_exp.head())

In [ ]:
reviews_annotated = pd.concat([reviews_eval, reviews_exp], ignore_index=True)

In [ ]:
reviews_annotated.value_counts('adjudicated')

In [ ]:
adjudicated = reviews_annotated[['text', 'adjudicated']].melt(
        id_vars=['text'], 
        value_vars=['adjudicated'], 
        var_name='adjudicated'
    ).rename(
        columns={'value': 'label'}
    )[['adjudicated', 'label', 'text']]

In [ ]:
adjudicated.to_csv('data/adjudicated.txt', sep='\t', header=False)

In [ ]:
individual_annotations = reviews_eval[['text', 'stalebi', 'zhossainzadeh']].reset_index().melt(
        id_vars=['index', 'text'],
        value_vars=['stalebi', 'zhossainzadeh'],
        var_name='annotator'
    ).sort_values(
        'index', 
        ignore_index=True
    ).rename(
        columns={'value': 'label'}
    )[['index', 'annotator', 'label', 'text']]

In [ ]:
individual_annotations

In [ ]:
individual_annotations.to_csv('data/individual_annotations.txt', sep='\t', index=False, header=False)

AP3:

In [9]:
ap3_annotated = pd.read_csv('data/3034029249.txt', sep='\t')

In [10]:
ap3_annotated

,data ID,annotator,label,text
0,522,3034029249,NaN,"Perfect in a steaming cup of cocoa, as a toppi..."
1,927,3034029249,NaN,Forty years of coffee-roasting expertise inspi...
2,696,3034029249,NaN,Wholesome Fair Trade Organic Cane Sugar is mad...
3,724,3034029249,NaN,Quality and value since 1930. Artificially fla...
4,716,3034029249,NaN,Nuts are the heroes of wholesome snacking. And...
...,...,...,...,...
95,510,3034029249,NaN,"Freshly milled on cool and slow turning, centu..."
96,643,3034029249,NaN,Pacific Foods Enriched Soy Beverage is great-t...
97,644,3034029249,NaN,Oven-Toasted Corn Cereal. Provides 12 essentia...
98,998,3034029249,NaN,It's funny how something as simple as a bowl o...


In [65]:
labels_dict = {
    1: 'health',
    2: 'versatility',
    3: 'gourmet experience',
    4: 'convenience',
    5: 'taste'
}

In [126]:
i = 99
ap3_annotated.loc[i]['text']

'This creamy, delicious milk chocolate with crunchy almonds and toffee is music to your mouth! The aptly named SYMPHONY Chocolate Bar makes a sweet holiday gift and a wonderful addition to baked treats.'

In [127]:
labels_val = [
    2,
    5,
    3,
    5,
    1,
    5,
    1,
    5,
    1,
    5,
    1,
    2,
    5,
    2,
    4,
    5,
    3,
    3,
    5,
    5,
    5,
    1,
    5,
    4,
    1,
    3,
    2,
    5,
    3,
    5,
    2,
    2,
    1,
    5,
    5,
    5,
    5,
    3,
    5,
    2,
    5,
    5,
    3,
    5,
    3,
    1,
    3,
    5,
    2,
    5,
    1,
    3,
    1,
    5,
    3,
    2,
    1,
    1,
    3,
    3,
    1,
    5,
    5,
    5,
    2,
    3,
    3,
    3,
    3,
    1,
    1,
    4,
    3,
    5,
    5,
    1,
    3,
    5,
    4,
    5,
    2,
    2,
    4,
    5,
    1,
    1,
    3,
    5,
    1,
    1,
    3,
    3,
    4,
    3,
    2,
    3,
    1,
    1,
    3,
    3
]

In [128]:
len(labels_val)

100

In [129]:
labels = [labels_dict[i] for i in labels_val]
labels

['versatility',
 'taste',
 'gourmet experience',
 'taste',
 'health',
 'taste',
 'health',
 'taste',
 'health',
 'taste',
 'health',
 'versatility',
 'taste',
 'versatility',
 'convenience',
 'taste',
 'gourmet experience',
 'gourmet experience',
 'taste',
 'taste',
 'taste',
 'health',
 'taste',
 'convenience',
 'health',
 'gourmet experience',
 'versatility',
 'taste',
 'gourmet experience',
 'taste',
 'versatility',
 'versatility',
 'health',
 'taste',
 'taste',
 'taste',
 'taste',
 'gourmet experience',
 'taste',
 'versatility',
 'taste',
 'taste',
 'gourmet experience',
 'taste',
 'gourmet experience',
 'health',
 'gourmet experience',
 'taste',
 'versatility',
 'taste',
 'health',
 'gourmet experience',
 'health',
 'taste',
 'gourmet experience',
 'versatility',
 'health',
 'health',
 'gourmet experience',
 'gourmet experience',
 'health',
 'taste',
 'taste',
 'taste',
 'versatility',
 'gourmet experience',
 'gourmet experience',
 'gourmet experience',
 'gourmet experience',
 'he

In [130]:
ap3_annotated['label'] = pd.Series(labels)
ap3_annotated

,data ID,annotator,label,text
0,522,3034029249,versatility,"Perfect in a steaming cup of cocoa, as a toppi..."
1,927,3034029249,taste,Forty years of coffee-roasting expertise inspi...
2,696,3034029249,gourmet experience,Wholesome Fair Trade Organic Cane Sugar is mad...
3,724,3034029249,taste,Quality and value since 1930. Artificially fla...
4,716,3034029249,health,Nuts are the heroes of wholesome snacking. And...
...,...,...,...,...
95,510,3034029249,gourmet experience,"Freshly milled on cool and slow turning, centu..."
96,643,3034029249,health,Pacific Foods Enriched Soy Beverage is great-t...
97,644,3034029249,health,Oven-Toasted Corn Cereal. Provides 12 essentia...
98,998,3034029249,gourmet experience,It's funny how something as simple as a bowl o...


In [131]:
ap3_annotated.to_csv('data/annotations.txt', sep='\t', index=False, header=False)

In [132]:
ap3_annotated.query('label == "gourmet experience"')

,data ID,annotator,label,text
2,696,3034029249,gourmet experience,Wholesome Fair Trade Organic Cane Sugar is mad...
16,667,3034029249,gourmet experience,"TERRA Real Vegetable Chips Original, Sea Salt,..."
17,559,3034029249,gourmet experience,"Pure white in color, C&H Powdered Sugar has a ..."
25,572,3034029249,gourmet experience,"SOY DREAM Enriched Original Organic Soymilk , ..."
28,740,3034029249,gourmet experience,All Natural Squeezable Applesauce on the Go. S...
37,783,3034029249,gourmet experience,The rumors are mounting. Word on the street? A...
42,995,3034029249,gourmet experience,Tostitos multigrain scoops. Tortilla chips are...
44,873,3034029249,gourmet experience,"The soft, chewy, gooey, unearthly goodness of ..."
46,754,3034029249,gourmet experience,"Organic wheat is boiled, sun-dried, cracked, a..."
51,647,3034029249,gourmet experience,Naturally fermented red wine is patiently ferm...
